In [2]:
import requests
import boto3
import json
import os
import sys
import argparse
import importlib
import transformers
import torch
import pathlib
import awswrangler as wr
from IPython.display import display
from sagemaker.huggingface.processing import HuggingFaceProcessor
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import FrameworkProcessor
from sagemaker.sklearn.estimator import SKLearn
from sagemaker.workflow.steps import ProcessingStep
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.session import get_execution_role


# Adding ../01_modules or ./01_modules to the system path so that we can load modules from 
# there as well
if '__file__' in globals():
    script_dir = pathlib.Path(__file__).parent.resolve()
else:
    script_dir = pathlib.Path().absolute()
modules_path_in_dev = os.path.abspath(os.path.join(script_dir, '..', '01_modules'))
modules_path_in_prod = os.path.abspath(os.path.join(script_dir, '01_modules'))
if os.path.exists(modules_path_in_dev):
    sys.path.append(modules_path_in_dev)
if os.path.exists(modules_path_in_prod):
    sys.path.append(modules_path_in_prod)


# # Jupyter only reads a local module the first time after 
# # kernel start. Re-running a cell with 
# # "from mymodulename import *" would not change
# # anything, even if the imported module has since changed.
# # As a workaround, we need to directly load the module, 
# # use importlib.reload to reload it and then import * 
import utils
_ = importlib.reload(utils)
import config
_ = importlib.reload(config) 


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
config.py loaded: v0.1
utils.py loaded: v0.2.12
utils.py loaded: v0.2.12
config.py loaded: v0.1


In [3]:
utils.pd_set_options(cols=500)

wr.athena.read_sql_query("""
SELECT
    *
 FROM 
     "01_raw".semanticscholar_s2orc_v2 -- v2 has an _ prefix, it is removed from dowstream models
 LIMIT 5
 """, '01_raw')

,corpusid,openaccessinfo,title,authors,body,bibliography
0,39964499,"{'disclaimer': 'This content is derived from https://arxiv.org/abs/1208.0130. ', 'externalids': {'medline': '23214774v1', 'mag': '2081684912', 'acl': None, 'doi': '10.1103/PhysRevE.86.051302', 'medrxiv': None, 'pubmedcentral': None, 'arxiv': '1208.0130'}, 'license': None, 'url': 'https://arxiv.org/abs/1208.0130', 'status': 'GREEN'}",Origin of rebounds with a restitution coefficient larger than unity in nanocluster collisions.,"[H. Kuninaka, H. Hayakawa]","{'text': ' I. INTRODUCTION Nanoclusters are technologically important for the construction of nanodevices. Because the size of nanoclusters is mesoscopic, thermodynamic properties of such materials are still not well understood [1], though the methods to make nanoclusters such as adiabatic expansion through a nozzle and a laser ablation technique are well established. [2] Dynamics of nanoclusters are extensively investigated from both scientific and technological interest. There are many nu...","{'text': 'AcknowledgmentsWe would like to thank N. V. Brilliantov, T. Kawasaki, S. Takesue, R. Murakami, and K. Saitoh for their valuable comments.Also, HK would like to thank Y. Wang and W. Lechner who gave him many advises for calculation of order parameters and how to use their calculation codes.Parts of numerical computation in this work were carried out at Yukawa Institute Computer Facility.This work was supported by the Grant-in-Aid for the Global COE Program ""The Next Generation of Ph..."
1,234595201,"{'disclaimer': 'This content is derived from https://pmc.ncbi.nlm.nih.gov/articles/PMC8100426. Its open-access license is CCBYNCSA.', 'externalids': {'medline': '33995511v1', 'mag': None, 'acl': None, 'doi': '10.11909/j.issn.1671-5411.2021.04.010', 'medrxiv': None, 'pubmedcentral': '8100426', 'arxiv': None}, 'license': 'CCBYNCSA', 'url': 'https://pmc.ncbi.nlm.nih.gov/articles/PMC8100426', 'status': None}",Prevention of self-harm through early detection of depression among the elderly with permanent pacemaker: a case report,"[Yu-Wei Chang, Ju-Yi Chen]","{'text': ' gnosis of sick sinus syndrome. Multiple somatic complaints, especially chronic back pain, had impacted the patient's sleep quality and daily life. The patient also had a diagnosis of general anxiety disorder, and she had been regularly visiting a psychiatric clinic for insomnia over the two years preceding this event. The patient occasionally expressed suicidal ideation but her family did not pay much attention to her suicidal verbalizations. At about 2:00 a.m., during the night...","{'text': 'ACKNOWLEDGMENTSThis study was supported by the Ministry of Science and Technology of Taiwan, China (MOST 108-2218-E-006-019 & MOST 109-2218-E-006-024).All authors had no conflicts of interest to disclose. Depression in older adults. A Fiske, J L Wetherell, M Gatz, Annu Rev Clin Psychol. 52009 . F M Kusumoto, M H Schoenfeld, C Barrett, 2018 ACC/AHA/HRS guideline on the evaluation and management of patients with bradycardia and cardiac conduction delay: a report of the American Col..."
2,14415362,"{'disclaimer': 'This content is derived from https://arxiv.org/abs/0705.0584. ', 'externalids': {'medline': None, 'mag': '2031777933', 'acl': None, 'doi': '10.1007/S00605-008-0535-3', 'medrxiv': None, 'pubmedcentral': None, 'arxiv': '0705.0584'}, 'license': None, 'url': 'https://arxiv.org/abs/0705.0584', 'status': 'CLOSED'}",Multidimensional continued fractions and a Minkowski function,[Giovanni Panti],"{'text': ' Preliminaries The nth order Farey set F n in the real unit interval [0, 1] is defined by recursion: one starts with F 0 = {0/1, 1/1} and obtains F n by adding to F n−1 all the Farey sums v 1 ⊕ v 2 = (a 1 + a 2 )/(b 1 + b 2 ) of two consecutive elements v i = a i /b i of F n−1 . The union of all the F n 's is the set of all rational numbers in [0, 1]. Analogously, by starting with B 0 = F 0 and replacing the Farey sum with the barycentric sum v 1 v 2 = (v 1 + v 2 

In [13]:
wr.athena.read_sql_query("""
SELECT
    corpusid AS id_semanticscholar,
    JSON_EXTRACT_SCALAR(CAST(openaccessinfo AS JSON), '$.externalids.mag') AS id_mag,
    JSON_EXTRACT_SCALAR(CAST(openaccessinfo AS JSON), '$.externalids.doi') AS id_doi,
    JSON_EXTRACT_SCALAR(CAST(openaccessinfo AS JSON), '$.externalids.arxiv') AS id_arxiv,
    title,
    JSON_EXTRACT_SCALAR(CAST(openaccessinfo AS JSON), '$.license') AS license,
    JSON_EXTRACT_SCALAR(CAST(openaccessinfo AS JSON), '$.url') AS source_url,
    JSON_EXTRACT_SCALAR(CAST(openaccessinfo AS JSON), '$.status') AS openaccess_status,
    JSON_EXTRACT_SCALAR(CAST(body AS JSON), '$.text') AS content_text,
    JSON_EXTRACT_SCALAR(CAST(body AS JSON), '$.annotations.paragraph') AS annotations_paragraph,
    JSON_EXTRACT_SCALAR(CAST(body AS JSON), '$.annotations.section_header') AS annotations_section_header
    --'x' AS "x"
 FROM 
     "01_raw".semanticscholar_s2orc_v2
 WHERE
     TRUE
 LIMIT 10
 """, '01_raw')

,id_semanticscholar,id_mag,id_doi,id_arxiv,title,license,source_url,openaccess_status,content_text,annotations_paragraph,annotations_section_header
0,234869482,3155033926,10.5772/INTECHOPEN.97307,<NA>,Thoracoabdominal Compartment Syndrome,CCBY,https://doi.org/10.5772/INTECHOPEN.97307,HYBRID,"\nIntroduction\n\nThe diaphragm is formed from a number of composite origins in the embryo. The most important is the ""septum transversum"", which is a thick mass of cranial mesenchyme that gives rise to parts of the thoracic diaphragm. Without dwelling into more details, the septum transversum merges with mesoderm surrounding the esophagus, the growing pleura and peritoneum ('pleuroperitoneal folds') and the growing muscles of the abdominal wall. The septum transversum gives rise to the cent...","[{""attributes"":null,""end"":612,""start"":15},{""attributes"":null,""end"":730,""start"":614},{""attributes"":null,""end"":999,""start"":732},{""attributes"":null,""end"":1550,""start"":1001},{""attributes"":null,""end"":1777,""start"":1552},{""attributes"":null,""end"":2191,""start"":1779},{""attributes"":null,""end"":2552,""start"":2193},{""attributes"":null,""end"":2851,""start"":2554},{""attributes"":null,""end"":3350,""start"":2880},{""attributes"":null,""end"":3706,""start"":3352},{""attributes"":null,""end"":3930,""start"":3708},{""attributes"":null...","[{""attributes"":{""n"":""1.""},""end"":13,""start"":1},{""attributes"":{""n"":""2.""},""end"":2878,""start"":2853},{""attributes"":{""n"":""3.""},""end"":5257,""start"":5201},{""attributes"":{""n"":""3.1""},""end"":5279,""start"":5259},{""attributes"":{""n"":""3.2""},""end"":6340,""start"":6286},{""attributes"":{""n"":""3.3""},""end"":9066,""start"":9029},{""attributes"":{""n"":""4.""},""end"":10473,""start"":10463}]"
1,270297560,<NA>,10.3390/dj12060170,<NA>,Periodontal Regeneration of Vital Poor Prognosis Teeth with Attachment Loss Involving the Root Apex: Two Cases with up to 5 Years Follow-Up,CCBY,https://pmc.ncbi.nlm.nih.gov/articles/PMC11202695,GOLD,"\nIntroduction\n\nPeriodontal regeneration is the ultimate goal of periodontal therapy and is defined as the ""restoration of lost or diminished periodontal tissues including cementum, periodontal ligament, and alveolar bone"" [1].Guided tissue regeneration (GTR), on the other hand, refers to a surgical procedure seeking to obtain the objectives of periodontal regeneration through utilising barrier devices or membranes, to exclude epithelial cells and provide space maintenance [1].The clinical...","[{""attributes"":null,""end"":1005,""start"":15},{""attributes"":null,""end"":3444,""start"":1007},{""attributes"":null,""end"":4719,""start"":3469},{""attributes"":null,""end"":5712,""start"":4721},{""attributes"":null,""end"":6359,""start"":5722},{""attributes"":null,""end"":9027,""start"":6369},{""attributes"":null,""end"":9648,""start"":9037},{""attributes"":null,""end"":10673,""start"":9650},{""attributes"":null,""end"":12286,""start"":10683},{""attributes"":null,""end"":12578,""start"":12297},{""attributes"":null,""end"":13477,""start"":12589},{""attr...","[{""attributes"":{""n"":""1.""},""end"":13,""start"":1},{""attributes"":{""n"":""2.""},""end"":3467,""start"":3446},{""attributes"":{""n"":""2.1.""},""end"":5720,""start"":5714},{""attributes"":{""n"":""2.1.""},""end"":6367,""start"":6361},{""attributes"":{""n"":""2.2.""},""end"":9035,""start"":9029},{""attributes"":{""n"":""2.2.""},""end"":10681,""start"":10675},{""attributes"":{""n"":""3.""},""end"":12295,""start"":12288},{""attributes"":{""n"":""3.""},""end"":12587,""start"":12580},{""attributes"":{""n"":""4.""},""end"":13489,""start"":13479},{""attributes"":{""n"":""4.""},""end"":142..."
2,251953573,<NA>,10.1103/PhysRevD.108.094029,2208.14827,Entanglement renormalization of the class of continuous matrix product states,CCBY,https://arxiv.org/abs/2208.14827,HYBRID,"\nTensor Network states are the entanglement-based ansatz that has arisen in recent years based on the renormalizati

In [17]:
utils.pd_set_options()
wr.athena.read_sql_query("""
WITH
raw_semanticscholar_s2orcv2 AS 
(
    SELECT * FROM "01_raw".semanticscholar_s2orc_v2 LIMIT 5000
),
base_semanticscholar_s2orcv2 AS 
(
    SELECT
        corpusid AS id_semanticscholar,
        JSON_EXTRACT_SCALAR(CAST(openaccessinfo AS JSON), '$.externalids.mag') AS id_mag,
        JSON_EXTRACT_SCALAR(CAST(openaccessinfo AS JSON), '$.externalids.doi') AS id_doi,
        JSON_EXTRACT_SCALAR(CAST(openaccessinfo AS JSON), '$.externalids.arxiv') AS id_arxiv,
        title,
        JSON_EXTRACT_SCALAR(CAST(openaccessinfo AS JSON), '$.license') AS license,
        JSON_EXTRACT_SCALAR(CAST(openaccessinfo AS JSON), '$.url') AS source_url,
        JSON_EXTRACT_SCALAR(CAST(openaccessinfo AS JSON), '$.status') AS openaccess_status,
        JSON_EXTRACT_SCALAR(CAST(body AS JSON), '$.text') AS content_text,
        JSON_EXTRACT_SCALAR(CAST(body AS JSON), '$.annotations.paragraph') AS annotations_paragraph,
        JSON_EXTRACT_SCALAR(CAST(body AS JSON), '$.annotations.section_header') AS annotations_section_header
        --'x' AS "x"
     FROM 
         raw_semanticscholar_s2orcv2
)
SELECT * FROM base_semanticscholar_s2orcv2 LIMIT 5
""", '01_raw')

,id_semanticscholar,id_mag,id_doi,id_arxiv,title,license,source_url,openaccess_status,content_text,annotations_paragraph,annotations_section_header
0,26516356,1522375164,10.1016/S0065-3527(08)60087-1,<NA>,Pathogenesis of Virus-Induced Demyelination,unspecified-oa,https://pmc.ncbi.nlm.nih.gov/articles/PMC7131186,GREEN,"\nI. INTRODUCTION\n\nA . Demyelinating Diseases of Humans\n\nDemyelination is the loss of myelin, the lipid sheath surrounding the neuronal axon. Myelin loss can occur following direct damage of the myelin (primary demyelination) or secondary to neuronal damage and axonal loss (secondary demyelination). Myelin is produced by oligodendrocytes in the central nervous system (CNS) and Schwann cells in the peripheral nervous system (PNS). The biochemistry of these two forms is different.\n\nDemyelination is a component of several viral diseases of humans. The best known of these are subacute sclerosing panencephalitis (SSPE) of children, produced by measles virus, and progressive multifocal leukoencephalopathy (PML), produced in adults and children by J C papovavirus. Both are rare conditions, SSPE being a late onset complication of measles and PML occurring in association with immunosuppression. In addition to these two well-described examples of viral demyelination there are several o...","[{""attributes"":null,""end"":482,""start"":56},{""attributes"":null,""end"":1402,""start"":484},{""attributes"":null,""end"":2380,""start"":1404},{""attributes"":null,""end"":3169,""start"":2382},{""attributes"":null,""end"":3750,""start"":3171},{""attributes"":null,""end"":5696,""start"":3794},{""attributes"":null,""end"":6952,""start"":5698},{""attributes"":null,""end"":7956,""start"":6954},{""attributes"":null,""end"":11482,""start"":7958},{""attributes"":null,""end"":12764,""start"":11484},{""attributes"":null,""end"":14212,""start"":12766},{""attributes"":null,""end"":15022,""start"":14260},{""attributes"":null,""end"":15630,""start"":15024},{""attributes"":null,""end"":15913,""start"":15632},{""attributes"":null,""end"":17206,""start"":15915},{""attributes"":null,""end"":18164,""start"":17208},{""attributes"":null,""end"":19592,""start"":18166},{""attributes"":null,""end"":20406,""start"":19594},{""attributes"":null,""end"":21218,""start"":20408},{""attributes"":null,""end"":21932,""start"":21220},{""attributes"":null,""end"":22222,""start"":21974},{""attributes"":null,""end"":23482,""start"":22224},{""at...","[{""attributes"":null,""end"":16,""start"":1},{""attributes"":null,""end"":54,""start"":18},{""attributes"":null,""end"":3792,""start"":3752},{""attributes"":null,""end"":14258,""start"":14214},{""attributes"":null,""end"":21972,""start"":21934},{""attributes"":{""n"":""11.""},""end"":26720,""start"":26684},{""attributes"":null,""end"":26747,""start"":26722},{""attributes"":null,""end"":40086,""start"":40067},{""attributes"":null,""end"":53469,""start"":53445},{""attributes"":null,""end"":65266,""start"":65251},{""attributes"":null,""end"":74486,""start"":74461},{""attributes"":null,""end"":79297,""start"":79247},{""attributes"":{""n"":""111.""},""end"":83021,""start"":82993},{""attributes"":null,""end"":83061,""start"":83023},{""attributes"":null,""end"":84638,""start"":84599},{""attributes"":null,""end"":86618,""start"":86481},{""attributes"":null,""end"":88423,""start"":88377},{""attributes"":null,""end"":90291,""start"":90253},{""attributes"":null,""end"":94586,""start"":94523},{""attributes"":null,""end"":95520,""start"":95499},{""attributes"":null,""end"":96494,""start"":96442},{""attributes"":null,""end"":9654..."
1,7628358,2271775406,10.4103/0301-4738.176038,<NA>,A case of giant nodular posterior scleritis mimicking choroidal malignancy,CCBYNCSA,https://pmc.ncbi.nlm.nih.gov/articles/PMC4784081,GOLD,"\nPosterior scleritis is often an under-recognized entity given its low incidence and variable clinical manifestations. It comprises 10% of all cases of scleritis and is associated with systemic diseases in up to o

In [10]:
utils.pd_set_options()
wr.athena.read_sql_query("""
WITH
base_openalex_works_reduced AS ()
grouped AS (
    SELECT
        --primary_topic_short_id,
        --primary_topic_display_name,
        primary_topic_subfield_short_id,
        primary_topic_subfield_long_id,
        primary_topic_subfield_display_name,
        primary_topic_field_short_id,
        primary_topic_field_long_id,
        primary_topic_field_display_name,
        primary_topic_domain_short_id,
        primary_topic_domain_long_id,
        primary_topic_domain_display_name,
        COUNT(*) AS c
     FROM
        openalex_works_reduced
     GROUP BY
        primary_topic_subfield_short_id,
        primary_topic_subfield_long_id,
        primary_topic_subfield_display_name,
        primary_topic_field_short_id,
        primary_topic_field_long_id,
        primary_topic_field_display_name,
        primary_topic_domain_short_id,
        primary_topic_domain_long_id,
        primary_topic_domain_display_name
)
SELECT 
    *
FROM
    grouped
ORDER BY
    primary_topic_domain_long_id,
    primary_topic_domain_short_id,
    primary_topic_field_long_id,
    primary_topic_field_short_id,
    primary_topic_subfield_long_id,
    primary_topic_subfield_short_id
 """, '01_raw')

,primary_topic_subfield_short_id,primary_topic_subfield_long_id,primary_topic_subfield_display_name,primary_topic_field_short_id,primary_topic_field_long_id,primary_topic_field_display_name,primary_topic_domain_short_id,primary_topic_domain_long_id,primary_topic_domain_display_name,c
0,<NA>,1102,Agronomy and Crop Science,<NA>,11,Agricultural and Biological Sciences,<NA>,1,Life Sciences,1
1,<NA>,1103,Animal Science and Zoology,<NA>,11,Agricultural and Biological Sciences,<NA>,1,Life Sciences,2
2,<NA>,1105,"Ecology, Evolution, Behavior and Systematics",<NA>,11,Agricultural and Biological Sciences,<NA>,1,Life Sciences,2
3,<NA>,1109,Insect Science,<NA>,11,Agricultural and Biological Sciences,<NA>,1,Life Sciences,1
4,<NA>,1304,Biophysics,<NA>,13,"Biochemistry, Genetics and Molecular Biology",<NA>,1,Life Sciences,1
5,<NA>,1310,Endocrinology,<NA>,13,"Biochemistry, Genetics and Molecular Biology",<NA>,1,Life Sciences,1
6,<NA>,1311,Genetics,<NA>,13,"Biochemistry, Genetics and Molecular Biology",<NA>,1,Life Sciences,2
7,<NA>,1312,Molecular Biology,<NA>,13,"Biochemistry, Genetics and Molecular Biology",<NA>,1,Life Sciences,3
8,<NA>,2403,Immunology,<NA>,24,Immunology and Microbiology,<NA>,1,Life Sciences,2
9,<NA>,1207,History and Philosophy of Science,<NA>,12,Arts and Humanities,<NA>,2,Social Sciences,1
